In [1]:
corpus = ['the woman is a wise queen',
          'the man is a wise president',
          'she is a pretty woman',
          'he is a strong man',
          'she is still young',
          'he is very old',
          'he is the current president of US',
          'the prince is a son of the king',
          'the princess is a daughter of the king',
          'a prince is a young man',
          'a princess is a young woman']

In [2]:
# 한문장씩 읽으면서 " " 으로 분류하여, 요소 추출.
def function1(corpus):    
    sentences = []
    for sentence in corpus:
        sentences.append(sentence.split())

    window_size = 2 # q2) s의 의미를 적고 그에 맞게 변수명 변경하기

    data = []
    for sentence in sentences:
        # 한문장씩 읽으면서 window size 기준으로 요소 추출하기.
        for idx, word in enumerate(sentence):
            # neighbor 가져오기. 
            for neighbor in sentence[max(idx - window_size, 0) : min(idx + window_size, len(sentence)) + 1] : 
                if neighbor != word:
                    data.append([word, neighbor])
    return data

In [3]:
import pandas as pd

data = function1(corpus)
df = pd.DataFrame(data, columns = ['input', 'label'])

In [4]:
df.head(10)

,input,label
0,the,woman
1,the,is
2,woman,the
3,woman,is
4,woman,a
5,is,the
6,is,woman
7,is,a
8,is,wise
9,a,woman


In [5]:
df.shape

(194, 2)

In [6]:
# q3) unique values 뽑아내기
def function2(corpus):
    words = []
    for text in corpus:
        for word in text.split(' '):
            words.append(word)
    words = set(words)
    return words

In [7]:
words = function2(corpus)
print(words)

{'son', 'wise', 'still', 'the', 'young', 'man', 'daughter', 'is', 'US', 'prince', 'strong', 'pretty', 'of', 'she', 'a', 'current', 'princess', 'he', 'president', 'queen', 'woman', 'very', 'old', 'king'}


In [8]:
# q4) one_hot_encoding 하기 위한 단어별 인덱스 부여
# 단어 인덱스를 이용하여, 나중에 0000000 배열에 그 인덱스에 1을 넣어주면
# one_hot_encoding 자동 생성! 
def function3(words):
    word2int = {}
    for i,word in enumerate(words):
        word2int[word] = i
    return word2int

In [9]:

word2int = function3(words)
print(word2int)

{'son': 0, 'wise': 1, 'still': 2, 'the': 3, 'young': 4, 'man': 5, 'daughter': 6, 'is': 7, 'US': 8, 'prince': 9, 'strong': 10, 'pretty': 11, 'of': 12, 'she': 13, 'a': 14, 'current': 15, 'princess': 16, 'he': 17, 'president': 18, 'queen': 19, 'woman': 20, 'very': 21, 'old': 22, 'king': 23}


In [10]:
import numpy as np

# q5) 앞서 말한 word unique index에 1을 넣어줌으로써, one hot encoding
def function4(word_index, ONE_HOT_DIM):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[word_index] = 1
    return one_hot_encoding

In [ ]:
# q6) Word2Vec을 tensorflow로 구현한 코드에서 ? 부분을 올바르게 채워넣기
# Skip Gram
# cross entropy 참고
# https://ratsgo.github.io/deep%20learning/2017/10/02/softmax/
# https://kevinthegrey.tistory.com/123

In [11]:
import tensorflow as tf

ONE_HOT_DIM = len(words)

X = [] # input word
Y = [] # target word

# window size고려해서 중심 단어 기준으로 주변 단어 추출된 df
# skip grame을 하기 위한 전처리 단계된 것이 df이다.
for x, y in zip(df['input'], df['label']):
    X.append(function4(word2int[x], ONE_HOT_DIM))
    Y.append(function4(word2int[y], ONE_HOT_DIM))

# convert X,Y to numpy arrays
X_train = np.asarray(X)
print("X_train",X_train)
Y_train = np.asarray(Y)
print("Y_train",Y_train)

# placeholders for X_train and Y_train
# 해당되는 단어의 위치값만 1인 one_hot_dim 차원의 x
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
# y는 window size로 추출된 단어 짝?
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# embedding dimension
EMBEDDING_DIM = 2

# hidden layer : represent word vector eventually
# x는 one hot encoding으로 되어있음으로 1과 곱해지는 값이 결국 단어 vector가 된다. 
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
# 출력 layer로 넘어 가기 위해 hidden node의 사이즈를 one_hot_dim으로 맞춰준다. 
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
output = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function : cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(output), axis=[1]))

# training
train = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

C:\Users\sora\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


X_train [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Y_train [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
# 모델
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 
# 10000번 epoch
iteration = 10000
for i in range(iteration):
    # input : X_train which is one hot encoded word
    # label : Y_train which is one hot encoded neighbor word
    sess.run(train, feed_dict={x: X_train, y_label: Y_train})
    if i % 2000 == 0:
        print('iteration '+ str(i) +' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  5.943717
iteration 2000 loss is :  2.5070548
iteration 4000 loss is :  2.4539084
iteration 6000 loss is :  2.4328113
iteration 8000 loss is :  2.4196827


In [13]:
# the hidden layer (W1 + b1) -> look up table
vectors = sess.run(W1 + b1)
vectors

array([[-1.0978762 , -2.1517684 ],
       [-0.1036793 , -1.490266  ],
       [ 0.23846161, -1.1268069 ],
       [ 0.19145775, -0.5982637 ],
       [ 0.6188953 , -1.4304115 ],
       [-0.6755203 , -1.8086963 ],
       [-1.2464038 , -2.448387  ],
       [-0.2885266 , -0.74118865],
       [-1.2662954 , -0.46191442],
       [-0.9836996 , -3.3101535 ],
       [ 0.41692686, -1.5898042 ],
       [ 0.5206425 , -1.5247324 ],
       [-1.1570241 , -0.02426124],
       [ 0.3532982 , -1.6635354 ],
       [ 0.584262  , -0.7910292 ],
       [-0.8408792 , -1.2953045 ],
       [-0.95394766, -3.2238941 ],
       [-0.2187311 , -2.0093484 ],
       [-1.817772  , -0.04998982],
       [-0.8047435 , -1.2670597 ],
       [-0.6581907 , -1.7914922 ],
       [ 0.3132609 , -0.47521514],
       [ 0.17851496, -1.2807887 ],
       [-3.5373764 , -0.78490555]], dtype=float32)

In [14]:
sess.close()

In [15]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

,word,x1,x2
0,son,-1.097876,-2.151768
1,wise,-0.103679,-1.490266
2,still,0.238462,-1.126807
3,the,0.191458,-0.598264
4,young,0.618895,-1.430411
5,man,-0.675520,-1.808696
6,daughter,-1.246404,-2.448387
7,is,-0.288527,-0.741189
8,US,-1.266295,-0.461914
9,prince,-0.983700,-3.310153


In [16]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

# word와 두개의 좌표값 불러오기 
for word, x1, x2 in zip(w2v_df['word'], w2v_df['x1'], w2v_df['x2']):
    ax.annotate(word, (x1,x2 ))
    
PADDING = 1.0
#  최소, 최대값으로 그래프 폭 결정.
# PADDING은 여유값 설정 . 
x1_axis_min = np.amin(vectors, axis=0)[0] - PADDING
x2_axis_min = np.amin(vectors, axis=0)[1] - PADDING
x1_axis_max = np.amax(vectors, axis=0)[0] + PADDING
x2_axis_max = np.amax(vectors, axis=0)[1] + PADDING
 
plt.xlim(x1_axis_min,x1_axis_max)
plt.ylim(x2_axis_min,x2_axis_max)
plt.rcParams["figure.figsize"] = (10,10)

plt.show()

<Figure size 640x480 with 1 Axes>